In [10]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Lambda, LSTM, TimeDistributed, Reshape, Bidirectional
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.losses import binary_crossentropy
from keras import backend as K
import matplotlib.pyplot as plt
from oldslidingWindow import read_data, segment_data_by_day, sliding_window
from oldCustom_layers import CustomPenaltyLayer
FILE_PATH = '../Processed Data/Aruba_17/processed_data.csv'

data_df = read_data(FILE_PATH)
daily_segments = segment_data_by_day(data_df)
# Use the first 10 days
daily_segments = daily_segments[:100]
# Find the average length of the segments
avg_length = 0
for segment in daily_segments:
    avg_length += len(segment)
avg_length /= len(daily_segments)
avg_length = int(avg_length)
print(avg_length)
window_size = avg_length
overlap_ratio = 0.5
windows = sliding_window(daily_segments, window_size=window_size, overlap_ratio=overlap_ratio)

# Prepare the data
windows = np.asarray([window.to_numpy() for window in windows])

# Split the data into training and testing sets
batch_size = 32
validation_split = 0.2
timesteps = window_size
print(timesteps)
input_dim = windows[0].shape[1]

7921
7921


In [11]:
# Create a new train-test split using the windows
window_train, window_val = train_test_split(windows, test_size=validation_split, shuffle=False)

# Prepare the input data for the model by concatenating the windows along the time axis
X_train = np.concatenate(window_train, axis=0)
X_val = np.concatenate(window_val, axis=0)

# Normalize the data using minMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train = X_train.reshape((-1, timesteps, input_dim))
X_val = X_val.reshape((-1, timesteps, input_dim))

latent_dim = 2
encoding_dim = 32

# ==================== ENCODER ====================
inputs = Input(shape=(timesteps, input_dim), name='encoder_input')
x = Bidirectional(LSTM(encoding_dim * 2, return_sequences=True))(inputs)
x = Bidirectional(LSTM(encoding_dim, return_sequences=False))(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
# z_mean is the mean of the latent space
# z_log_var is the variance of the latent space

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# ================= LATENT SPACE ==================
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
# ==================== DECODER ====================
x = Dense(timesteps * encoding_dim, activation='relu')(latent_inputs)
x = Reshape((timesteps, encoding_dim))(x)
x = Bidirectional(LSTM(encoding_dim, return_sequences=True, input_shape=(timesteps, encoding_dim)))(x)
x = TimeDistributed(Dense(input_dim))(x)
# LSTM layer in the decoder is used to reconstruct the original sequence

# the VAE model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = Model(latent_inputs, x, name='decoder')
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

# VAE loss function with custom_penalty
reconstruction_loss = K.mean(K.square(inputs - outputs))
reconstruction_loss *= timesteps * input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

# Add the custom penalty to the loss function
penalty_weight = 10.0  # Adjust the weight of the penalty term as needed
penalty_layer = CustomPenaltyLayer(scaler, input_dim, timesteps)
penalty = penalty_layer(outputs)
penalty *= penalty_weight

vae_loss = K.mean(reconstruction_loss + kl_loss + penalty)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

num_epochs = 100
history = vae.fit(X_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, None))

# Use the encoder to generate embeddings for each sequence
encoder_model = Model(inputs, z_mean)

# Use the encoder to generate embeddings for each sequence
encoder_model = Model(inputs, z_mean)
X_embedded = encoder_model.predict(X_train, batch_size=batch_size)

Epoch 1/100
2/2 [==============================] - 54s 21s/step - loss: 31521.2344 - val_loss: 36155.4766
Epoch 2/100
2/2 [==============================] - 38s 18s/step - loss: 32155.5742 - val_loss: 35255.3906
Epoch 3/100
2/2 [==============================] - 37s 18s/step - loss: 30358.3418 - val_loss: 34029.6680
Epoch 4/100
2/2 [==============================] - 39s 19s/step - loss: 28813.3477 - val_loss: 32356.7559
Epoch 5/100
2/2 [==============================] - 36s 17s/step - loss: 27002.0684 - val_loss: 30696.1152
Epoch 6/100
2/2 [==============================] - 36s 18s/step - loss: 24552.0352 - val_loss: 26231.7930
Epoch 7/100
2/2 [==============================] - 40s 19s/step - loss: 21144.7305 - val_loss: 22735.4141
Epoch 8/100
2/2 [==============================] - 40s 20s/step - loss: 16520.3047 - val_loss: 17333.1777
Epoch 9/100
2/2 [==============================] - 42s 19s/step - loss: 15189.7881 - val_loss: 13820.0293
Epoch 10/100
2/2 [============================

In [12]:
# Generate a fake dataset using the VAE model
n_samples = len(windows)

# Sample from the latent space
z_samples = np.random.normal(size=(n_samples, latent_dim))

# Use the decoder to generate the output
predicted_values = decoder.predict(z_samples)
predicted_values = np.reshape(predicted_values, (n_samples, timesteps, input_dim))

# Undo the normalization
predicted_values = np.reshape(predicted_values, (-1, input_dim))
predicted_values = scaler.inverse_transform(predicted_values)

# Round each of the values in the array to the nearest integer
predicted_values = np.rint(predicted_values)

# Create the fake dataset in the original format
fake_dataset = []
for window in predicted_values.reshape((n_samples, timesteps, input_dim)):
    fake_dataset.extend(window)

# Save the fake dataset to a new file 'fake_dataset.txt'
fake_data = pd.DataFrame(fake_dataset, columns=['Date', 'Time', 'Device ID', 'Status', 'Activity', 'Activity Status'])

with open('Aruba_17_prediction.txt', 'w') as file:
    for _, row in fake_data.iterrows():
        file.write(','.join(map(str, row.values)) + '\n')

2/2 [==============================] - 2s 450ms/step
